![here](https://miro.medium.com/max/1400/1*ziDWjvLlK9acS0SJYHB2Fw.png)

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
import gensim
import scipy
from scipy import spatial
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 


<p id="toc"></p>

<br><br>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #3eb489; background-color: #ffffff;">TABLE OF CONTENTS</h1>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#jaccard-similarity">1.&nbsp;&nbsp;&nbsp;&nbsp;Jaccard Similarity</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#cosine-similarity">2.&nbsp;&nbsp;&nbsp;&nbsp;Cosine Similarity</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#word-mover-distance">3.&nbsp;&nbsp;&nbsp;&nbsp;Word Mover Distance</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#variational-autoencoder">4.&nbsp;&nbsp;&nbsp;&nbsp;Variational AutoEncoder</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#pre-trained-sentence-encoder">5.&nbsp;&nbsp;&nbsp;&nbsp;Pre-trained Sentence Encoder</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#siamese-manhattan-lstm">6.&nbsp;&nbsp;&nbsp;&nbsp;Siamese Manhattan LSTM (MaLSTM)</a></h3>

---

<br>

<a id="jaccard-similarity"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #3eb489; background-color: #ffffff;" id="jaccard-siilarity">1.&nbsp;&nbsp;JACCARD SIMILARITY&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---


<p>Jaccard similarity is length of intersection of two sets divided by length of union of two sets. This measurement refers to number of common words in both sets.More number of common words means both objects should be similarity.<p/><p>But we must apply some preprocessing before using jaccard and keep that sentence in form array containing tokens of that sentence</p>


In [ ]:
def jaccard(array1, array2):
    intersection = set(array1).intersection(set(array2))
    union = set(array1).union(set(array2))
    return len(intersection) / len(union)

def preprocess(text):
    array = []
    text = re.sub("[^a-zA-z]", " ",text)
    text = text.lower()
    text = word_tokenize(text)
    for word in text:
        if word not in stop_words:
            array.append(word)
    return array

#example
sent1 = "AI is our friend and it has been friendly"
sent2 = "AI and humans have always been friendly"

stop_words = set(stopwords.words("english"))
array1 = preprocess(sent1) 
array2 = preprocess(sent2)
jaccard(array1,array2)

<p>Let's apply jaccard similarity on our dataset</p>

In [ ]:
def stemming(array):
    new_array = []
    for word in array:
        new_array.append(stemmer.stem(word))
    return new_array

def lemma(array):
    new_array = []
    for word in array:
        new_array.append(lemmatizer.stem(word))
    return new_array
stemmer = PorterStemmer()
df= pd.read_csv("../input/us-patent-phrase-to-phrase-matching/train.csv")

df["pr_anchor"] = df["anchor"].apply(preprocess)
df["pr_target"] = df["target"].apply(preprocess)

df["pr_stem_anchor"] = df["pr_anchor"].apply(stemming)
df["pr_stem_target"] = df["pr_target"].apply(stemming)


df["jaccard_score"] = df.apply(lambda x: jaccard(x["pr_stem_anchor"], x["pr_stem_target"]), 1)
df.head()

<p>But there is some problem</p>
1.what if two sentence has same meaning but don't have any common words.

        For example :
        sent1 = "President greets the press in Chicago"
        sent2 = "Obama speaks in Illinois"
        
2.What if two sentence has common words but don't have any similarity

        For example:
        sent1 = "Play the record"
        sent2 = "Record the play"

In [ ]:
#Let's see jaccard score for case 1. 
sent1 = "President greets the press in Chicago"
sent2 = "Obama speaks in Illinois"

array1 = preprocess(sent1)
array2 = preprocess(sent2)
print("jccard_score for case one = ",jaccard(array1, array2))

#Let's see jaccard score for case 2. 
sent3 = "Play the record"
sent4 = "Record the play"

array3 = preprocess(sent3)
array4 = preprocess(sent4)
print("jccard_score for case two = ",jaccard(array1, array2))

<p>see for our first case score is 0.0 . This is terrible distance even two sentences have similar meaning. and score for second case is this is terrible too even two sentence has no similar meaning. In our case one jaccard simlilarity niether able to capture lexial similarity nor semantic similarity. This is because jaccard similary work like <b>"more common words more similarity no common words no similarity"</b></p>

<br>

<a id="cosine-similarity"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #3eb489; background-color: #ffffff;" id="cosine-similarity">2.&nbsp;&nbsp;Cosine Similarity&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---


<p>cosine similarity is the cosine angle between two vector.</p>

![here](https://miro.medium.com/max/426/1*5J8YlnfnZlzFobQC9cGk-w.png)

<p>Mathematically speaking cosine similarity is the measure of similarity between two non-zero vector in <b>inner-product space</b> which measure the cosine angle between them.The cosine of 0º is 1 and it is less than 1 for any angle between (0,π] radians.Two vectors with same orientation has similarity 1, two vector oriented at 90º has similarity 0 and two vectors with 180º has similarity of -1 independent of their magnitude</p>

<br>

<a id="pretrained-embedding-cosine-similarity"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #3eb489; background-color: #ffffff;" id="pretrained-embedding-cosine-similarity">2.1&nbsp;&nbsp;Pretrained Embedding (Glove) + Cosine Similarity&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---


In [ ]:
glove_file_path = "../input/glove-embeddings/glove.6B.300d.txt"
embeddings_index = {}
with open(glove_file_path, "r") as f:
    for line in f:
        values = line.rstrip().rsplit(" ")
        word = values[0]
        coef = np.asarray(values[1:], dtype="float")
        embeddings_index[word] = coef

In [ ]:
def cosine_similarity(array1, array2):
    try:
        vector1 = np.mean([embeddings_index[word] for word in array1])
        vector2 = np.mean([embeddings_index[word] for word in array2])
    except KeyError:
        return 0.0
    
    cosine = spatial.distance.cosine(vector1, vector2)
    return 1 - cosine

In [ ]:
df["pr_anchor"] = df["anchor"].apply(preprocess)
df["pr_target"] = df["target"].apply(preprocess)
df["glove_cosine"] = df.apply(lambda x: cosine_similarity(x["pr_anchor"], x["pr_target"]), 1)
df.head()

<br>

<a id="smooth-inverse-frequency"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #3eb489; background-color: #ffffff;" id="smooth-inverse-frequency">2.2&nbsp;&nbsp;Smooth Inverse Frequency&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---


<p>Taking average of the word embeddings in sentence tends to give too much weight to the irrelevant word. Smooth Inverse frequency try to solve this problem by following way</p>
<p><b>1.Weighted Average</b></p>
<p>This take weighted average of every word embedding by a/(a + p(w)) where a is ususally 1e-3 and p(w) is the freaquency of that word in that sentence.</p>
<p><b>2.Commomn Component removal</b></p>
<p>This computes principal component of resulting set of senetence embeddings. And subtract it from those sentence embedding</p>
<p> SIF downgrades unimportant words such as "just","but" etc. and keep the information which contributes most to the semantic of the sentence. But we are not going to apply smooth inverse frequency because in our data we dont have big sentences.

<br>

<a id="word-mover-distance"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #3eb489; background-color: #ffffff;" id="word-mover-distance">3.&nbsp;&nbsp;Word Mover's Distance&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---


<p><h2>Earth Mover Distance</h2></p>
<p>&nbsp;</p>
<p>The core part of WMD is earth mover distance (EMD).So fisrt i want to share idea of EMD.</p>
<p>EMD solve transportation problem. For instance we have m and n while m and n derives set of suppliers and warehouse. The target is going to minimize transportation cost such that shipping all goods from m to n.Given that there are contrains:</p>
<p>1.only allow transportation from m to n. Not allowing transporattion from n to m.</p>
<p>2.Total number of sending cargoes doesn't exceed total capacity of m.</p>
<p>3.Total number of recieving cargoes doesn't exceed total capacity of n.</p>
<p>4.Total number of transportation is the minimum between total cargoes in m and total cargoes in n.</p>
<p>&nbsp;</p>
<p><h4>The denotations are :</h4></p>
<p>* p: Set of origin</p>
<p>* q: Set of destination</p>
<p>* f(i, j): flow from i to j</p>
<p>* m: number of origin</p>
<p>* n: number of destination</p>
<p>* w(i, j): Number of cargoes transport from i to j</p>
<p>&nbsp;</p>
<img src="https://miro.medium.com/max/558/1*T1ExbOo5LooayRrIcpPrbA.png" />
<p>&nbsp;</p>
<p><h2>Word Mover's Distance</h2></p>
<p>&nbsp;</p>
<p>WMD is designed to overcame synonym problem</p>
<p>The typical example is</p>
<p>sent1: "Obama speaks to the media in Illinois"</p>
<p>sent2: "The president greets the press in Chicago"</p>
<p>Except the stopwords there is no common words among two sentence but both of them taking about same topic(at the time).</p>
<img src="https://miro.medium.com/max/890/1*nTWAm46JMYWXpHVsS9MA5w.png" />
<p>&nbsp;</p>
<p>Retrive vectors from any pretrained embedding that can be glove,fasttext, word2vec or custom embedding. After that it is using nBOW to represent weight or importance. It allows words to transfer from sentence1 to sentence2 because alogorithm doesn't know "obama" would transfer to president. At the end it will choose the minimum transportation cost to transfer each word from sent1 to sent2. </p>

In [ ]:
#Let's see how WMD works on our cases
fasttext_file_path = "../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec"
fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(fasttext_file_path)
score1 = fasttext_model.wmdistance(array1,array2)
print(score1)
score2 = fasttext_model.wmdistance(array3,array4)
print(score2)

In [ ]:
#Let's apply WMD on our dataset
def wmd(array1,array2):
    score = fasttext_model.wmdistance(array1,array2)
    return score

df["WMD"] = df.apply(lambda x: wmd(x["pr_anchor"],x["pr_target"]), 1)
df.head()

<br>

<a id="variational-autoencoder"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #3eb489; background-color: #ffffff;" id="variational-autoencoder">4.&nbsp;&nbsp;Variational AutoEncoder&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---


<p><b>What are Autoencoders?</b></p>
<p>Autoencoders are a special type of neural network architecture in which output is same as input. Autoencoders are trained in an unsupervised manner in order to learn the extremly low level representations of the input data. These low level features are then deformed by decoder to project actual data. An autoencoder is a regression task where network is asked to predict it's input. These network has tight bottleneck of few neuron in middle forcing them to create effective represention of that compress input that can use by decoder to reproduce orignal input.</p>
<p>&nbsp;</p>
<img src="https://miro.medium.com/proxy/1*HIBRgSV2ePFtCSOF_lgPCQ.png">
<p>&nbsp;</p>
<p>An autoencoder comprise of three main component:</p>
<p><b>• Encoder Architecture :</b> The encoder architecture is series of layers with decreseing number of nodes and ultimately reduce to latent view representation</p>
<p><b>• Latent View Representation:</b> The Latent view represent lowest level space in which input is reduced and information is preserved</p>
<p><b>• Decoder Architecture :</b> The decoder architecture is series of layers with increasing number of nodes and ultimately outputs similer(almost) input.</p>
For more in depth infomation you can read this <a href="https://www.kaggle.com/code/shivamb/how-autoencoders-work-intro-and-usecases/notebook" target="_top">article</a>

<br>

<a id="pre-trained-sentence-encoder"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #3eb489; background-color: #ffffff;" id="pre-trained-sentence-encoder">5.&nbsp;&nbsp;Pre-trained Sentence Encoders&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---


<p>The universal sentence encoders encode text into high dimensional vectors that cn be used for text classification, semantic similarity, clustering and other natural language task.</p>
<p>Pretrained sentence encoders aim to play same role as word2vec and Glove but for senetence embeddings: The embeddings they produce can be used in variety of application, such as text classification, paraphrase detection, etc. Typically they have trained on a range of supervised and unsupervised task, in order to capture as much universal semantic information as possible.</p>
<p>&nbsp;</p>
<img src="https://miro.medium.com/max/700/0*sYubQH60abTnSnAq.png">
<p>&nbsp;</p>
<p>Several such encoders are available :</p>
<p><b>• Infersent(Facebook Research)</b>: BiLSTM with max pooling, trained on SNLI dataset, 570k, English sentence pairs labelled with one of three categories: entailment, contradiction or neutral.</p>
<p><b>• Google Sentence Encoder</b>: a simpler Deep Averaging Network where input embeddings for words and bigrams are avarged together and passed through a feed-forward deep neural network.</p>

In [ ]:
cd ../input/infersent-models

In [ ]:
from models import InferSent

def cosine_similarity(text1,text2):
    try :
        array1 = model.encode([text1])[0]
        array2 = model.encode([text2])[0]
        cosine = scipy.spatial.distance.cosine(array1,array2)
    except KeyError:
        return 0.50
    return 1 - cosine


model_version = 1
model_path = f"../infersent1-2/infersent{model_version}.pkl"
params_model = {
    "bsize":64,
    "word_emb_dim":300,
    "enc_lstm_dim":2048,
    "pool_type":"max",
    "dpout_model":0.0,
    "version":model_version
}

model = InferSent(params_model)
model.load_state_dict(torch.load(model_path))

use_cuda = False
model = model.cuda() if use_cuda else model

glove_path = "../glove-embeddings/glove.6B.300d.txt"
model.set_w2v_path(glove_path)
model.build_vocab_k_words(K=100000)

df["infersent_cosine"] = df.apply(lambda x: cosine_similarity(x["anchor"],x["target"]), 1)

<br>

<a id="siamese-manhattan-lstm"></a>

<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: #3eb489; background-color: #ffffff;" id="siamese-manhattan-lstm">6.&nbsp;&nbsp;Siamese Manhattan LSTM (MaLSTM)&nbsp;&nbsp;&nbsp;&nbsp;<a href="#toc">&#10514;</a></h1>

---


<p>Siamese networks are networks that have <b>two or more identical sub-networks in them</b>.Siamese network seems to <b>perform well on similarity tasks</b> and have been used for task like sentence semantic similarity , recoonizing forged signatures and many more.</p>
<p>The name MaLSTM  and SiameseLSTM might leave and impression that there are some kind of new LSTM unit proposed, but that not the case.</p>
<p><b>Siamese</b> is the name of the general model architecture where the model consist of two identical subnetworks that compute some kine of representation vectors for two inputs and a distance measure is used to compute a score to estimate the similarity or difference of the inputs. In the attached figure the LSTMa and LSTMb share parameters(weights) and have identical structure. The idea itself is not new and goes back to 1994.</p>
<p><b>MaLSTM</b> (ManhattanLSTM) just refers to the fact that they chose to use Manhattan distance to compare the final hidden state of two standard LSTM layers. Alternatives like cosine and Euclidean distance can also be used but the auther states that "Manhattan distance slighty outperforms other reasonable alternatives such as cosine similarity".</p>
<p>&nbsp;</p>
<img src="https://miro.medium.com/max/602/0*_km0_7zWh8crsoc-">
<p>&nbsp;</p>
<p>&nbsp;</p>